# Variable transformation steps for TOPOFLOW

This notebook steps through the needed transformation for meteorological data in the NetCDF format to the input used by TOPOFLOW

## Table of Contents
1. [Meteorological Data](#data)  
    1.1. [Provenance](#provenance)  
    1.2. [Extracting variables from NetCDF](#openNetCDF)
2. [Dealing with time coordinates](#time)    
3. [Calculating needed variables from other variables](#calc)   
    3.1. [Relative Humidity](#RH)  
    3.2. [Wind Speed](#WS)  
4. [Units conversion](#units)  
    4.1. [Surface pressure](#pressure)  
    4.2. [Precipitation](#precip)
5. [Format conversion](#format)  

## <a name='data'> Meteorological data </a>

### <a name='provenance'> Provenance </a>

The meteorological comes from the [ECMWF ERA 20C](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era-20c), which can be accessed [here](http://apps.ecmwf.int/datasets/data/era20c-daily/levtype=sfc/type=fc/)

The data was downloaded on the 3-hr timestep (1 datapoint per 24hours) for the months of October, November, and December 2010. The files are stored in the GitHub directory.

### <a name='openNetCDF'> Extracting variables from NetCDF </a>

Details about NetCDFs file can be found [here](https://github.com/khider/netCDFTutorial/blob/master/Opening%20and%20reading%20NetCDF%20and%20GRIB%20file.ipynb).

**Step1**: Get the keys from the file itself. Ideally we want to query the data

In [1]:
from netCDF4 import MFDataset
# Just get a list of netCDF files. 
root = "/Volumes/Data HD/Documents/MINT/Climate/netCDFTutorial"
files = ["Oct2010.nc","Nov2010.nc","Dec2010.nc"]

file_names =[]
for name in files:
    file_names.append(root+"/"+name)
    
#Open the file and get the keys for this example
nc_fid = MFDataset(file_names)
keys=[]
nc_vars = [var for var in nc_fid.variables]
for vars in nc_vars:
    keys.append(getattr(nc_fid.variables[vars],'long_name'))

**Step2:** Dump the content of the NetCDF file into a Python dictionary.

In [2]:
def getMFNcVar(nc_files, keys):
    ''' Extract variables from a dataset across multiple netCDF files.
    
    This function gets the variable contained in a netCDF file 
    and return them into Python nested dictionaries. The first
    dictionary's key contains the longname, while the
    second dictionary contains values, standard name (CF),
    units and the missing data flag.
    
    Args:
        nc_files (list): A list of netCDF files containing the dataset
        keys (list): A list of keys to fetch the variables according
            to the CF standard
    
    Returns:
        dict_out (dict): A dictionary containing the standard names as keys and
            the associated data as values.
    '''
    # Import the package
    from netCDF4 import MFDataset
    # Open the netCDF files
    nc_fid = MFDataset(nc_files)
    # Get the variable names
    nc_vars = [var for var in nc_fid.variables]
    
    #Make empty lists to collect the info
    #longname (should be using the CF conventions)
    nc_vars_longname=[]
    #Units
    nc_vars_units=[]
    # Get the standard name
    nc_vars_standardname=[]
    #Corrections
    nc_vars_scale_factor=[]
    nc_vars_add_offset=[]
    #Missing values
    nc_vars_missing_value=[]
    
    for vars in nc_vars:
        if 'long_name' in nc_fid.variables[vars].ncattrs():
            nc_vars_longname.append(getattr(nc_fid.variables[vars],'long_name'))
        else:
            nc_vars_longname.append(vars)
        if 'units' in nc_fid.variables[vars].ncattrs():
            nc_vars_units.append(getattr(nc_fid.variables[vars],'units'))
        else:
            nc_vars_units.append('NA')
        if 'standard_name' in nc_fid.variables[vars].ncattrs():
            nc_vars_standardname.append(getattr(nc_fid.variables[vars],'standard_name'))
        else:
            nc_vars_standardname.append("NA")    
        if 'scale_factor' in nc_fid.variables[vars].ncattrs():
            nc_vars_scale_factor.append(getattr(nc_fid.variables[vars],'scale_factor'))
        else:
            nc_vars_scale_factor.append(1)
        if 'add_offset' in nc_fid.variables[vars].ncattrs():
            nc_vars_add_offset.append(getattr(nc_fid.variables[vars],'add_offset'))
        else:
            nc_vars_add_offset.append(0) 
        if 'missing_value' in nc_fid.variables[vars].ncattrs(): 
            nc_vars_missing_value.append(getattr(nc_fid.variables[vars],'missing_value'))
        else:
            nc_vars_missing_value.append('NA')
    # Check for the list against the desired variables and output.
    dict_out ={}
    for name in nc_vars_longname:
        if name in keys:
            f = {'values':[],'units':[],'missing_value':[], 'standard_name':{}}
            idx = nc_vars_longname.index(name)
            f['values']=(nc_fid.variables[nc_vars[idx]][:]*nc_vars_scale_factor[idx])\
                +nc_vars_add_offset[idx]
            f['units']=nc_vars_units[idx]
            f['missing_value'] = nc_vars_missing_value[idx]
            f['standard_name'] = nc_vars_standardname[idx]
            dict_out[name] = f
    
    return dict_out            

# Example
dict_out=getMFNcVar(file_names,keys)

#View the content of the file
dict_out

{'10 metre U wind component': {'missing_value': -32767,
  'standard_name': 'NA',
  'units': 'm s**-1',
  'values': array([[[-2.59051159, -2.59051945, -2.59052728, ..., -2.59071312,
           -2.59071436, -2.59071559],
          [-2.5905114 , -2.59051779, -2.59052422, ..., -2.59070276,
           -2.59070523, -2.59070767],
          [-2.59051201, -2.59051721, -2.59052244, ..., -2.59069344,
           -2.59069692, -2.59070036],
          ...,
          [-2.59074356, -2.59074239, -2.59074122, ..., -2.59175969,
           -2.59178496, -2.5918129 ],
          [-2.59075051, -2.59075044, -2.59075041, ..., -2.59175161,
           -2.59177928, -2.59180874],
          [-2.59075746, -2.59075853, -2.59075957, ..., -2.59174482,
           -2.5917746 , -2.59180549]],
  
         [[-2.59081699, -2.59082946, -2.590842  , ..., -2.59071712,
           -2.59071432, -2.59071156],
          [-2.59080517, -2.59081878, -2.59083235, ..., -2.59070825,
           -2.59070627, -2.59070429],
          [-2.590794

## <a name='time'> Dealing with time coordinates </a>

Time coordinate values pose a special challeneg to netCDF users. Most metadata standards (such as CF) specify that time should be measured relative to a fixed data using a certain calendar, with units specified like **hours since YY-MM-DD hh:mm:ss**. These units can be awkward to deal with, without a utility to convert the valuesto and from calendar dates. The function called `num2date` and `date2num` are provided with the `netCDF4` package to do just that.

`num2date` converts numeric values of time in the specified **units** and **calendar** to datetime objects, and `date2num` does the reverse.

- `units`: a string of the form **&lt;time units&gt; since &lt;reference time&gt;** describing the time units. **&lttime units&gh;** can be days, hours, minutes, seconds, milliseconds, or microseconds. **&lt;reference time&gt;** is the time origin.
- `calendar`: describes the calendar used in the time calculations. All the values currently defined in the [CF metadata convention](http://cfconventions.org/) are valid calendars: **'standard'**, **'gregorian'**, **'proleptic_gregorian'**, **'noleap'**, **'365_day'**, **'360_day'**, **'julian'**, **'all_leap'**, **'366_day'**. Default is **'standard'**, which is a mixed Julian/Gregorian calendar.

([source](http://unidata.github.io/netcdf4-python/#section7))

In [3]:
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num

dates = num2date(dict_out['time']['values'][:],dict_out['time']['units'])

## <a name='calc'> Calculating needed variables from other variables </a>

TOPOFLOW requires information about relative humidity and wind speed, which are not variables contained in the NetCDF file but can be calculated from other variables.

### <a name='RH'> Relative Humidity </a>

Relative humidity can be calculate from temperature and dewpoint temperature. In the query, care should be taken as to choose temperature taken at the same elevation. In this particular example, we use `2 metre temperature` and `2 metre dewpoint temperature`.

Relative humidity can then be calculated as follows:

$RH=100\times\frac{e^{\frac{17.625\times TD}{243.04+TD}}}{e^{\frac{17.625\times T}{243.04+T}}}$

where T (temperature) and TD (dewpoint temperature) are in deg C.

In [4]:
import numpy as np

TD = np.array(dict_out['2 metre dewpoint temperature']['values'])
T = np.array(dict_out['2 metre temperature']['values'])
RH = 100*(np.exp((17.625*TD)/(243.04+TD))/np.exp((17.625*T)/(243.04+T)))

Place the new variables in dict_out:

In [5]:
dict_out['relative humidity']={'values':RH,'units':'NA'} 

### <a name='WS'> Wind Speed </a>

Wind speed can be calculated from the u-wind and v-wind components. TOPOFLOW also calls for the reference height which is contained in the name of the variables: `10 metre U wind component` and `10 metre V wind component`.

Wind speed can be calculated as follows:

$WS = \sqrt{U^{2}+V^{2}}$

In [6]:
U = np.array(dict_out['10 metre U wind component']['values'])
V = np.array(dict_out['10 metre V wind component']['values'])

W = np.sqrt(U**2+V**2)

Place the new variables in dict_out:

In [7]:
dict_out['wind speed']={'values':W,'units':dict_out['10 metre V wind component']['units']} 

## <a name='units'> Units Conversion </a>

### <a name='pressure'> Surface Pressure </a>

ECMWF ERA 20C expresses pressure in Pa, while TOPOFLOW takes the input in mbar.

In [8]:
P = dict_out['Surface pressure']['values']/100

Put into the dict_out dictionary and add a note for conversion

In [9]:
dict_out['Surface pressure_converted']={'values':P,'units':'mbar','notes':'converted from "surface pressure"'}

### <a name='precip'> Precipitation </a>

Precipitation is given as total amount in the reanalysis dataset.

In [10]:
dict_out['Total precipitation']['units']

'm'

TOPOFLOW takes a rate input of mm/hr, which requires knowing the time step of the data (in this case, 3hr).

In [11]:
import numpy as np

precip_rate = 1000*np.array(dict_out['Total precipitation']['values'])/3

Place it back into the dictionary. Note that `Precipitation rate` would need to be replaced by

In [12]:
dict_out['Precipitation rate'] = {'values':precip_rate.tolist(), \
                                  'units':'mm/hr','notes':\
                                  'converted from amount, Total precipitation'}

## <a name='format'> Format conversion </a>

TOPOFLOW uses a binary grid to store the data for use by the software. Information [here](https://www.dropbox.com/s/n9bkpn8h8zri2i0/Peckham_et_al_2017_GPF_Appendices.pdf?dl=0)

The example below moves `Precipitation rate` to the file.

In [13]:
print(type(precip_rate))
print(precip_rate.dtype)
precip_rate=np.float32(precip_rate)
print(precip_rate.dtype)

<class 'numpy.ndarray'>
float64
float32


In [14]:
# Create a new grid file and save
precip_rate_file = 'precip_rate.txt'
grid_unit = open(precip_rate_file, 'wb')
precip_rate.byteswap(True)
precip_rate.tofile(grid_unit)
grid_unit.close()